# 第3章: ツールを使うシングルエージェント

## 準備

以下のセルを順番に実行して、演習に必要な環境をセットアップします。

### ライブラリのインストール

このセルは、LangGraphおよび関連するLangChainライブラリをインストールします。実行には数分かかる場合があります。
ご利用になるLLMプロバイダーに応じて、コメントアウトを解除して必要なライブラリをインストールしてください。

In [ ]:
# === ライブラリのインストール ===
# ご利用になるLLMプロバイダーに応じて、以下のコメントを解除して実行してください。
# !%pip install -qU langchain langgraph langchain_openai langchain_google_vertexai langchain_google_genai langchain_anthropic langchain_aws boto3

### LLMプロバイダーの選択

このセルでは、使用するLLMプロバイダーを選択します。
`LLM_PROVIDER` 変数に、利用したいプロバイダー名を設定してください。
選択可能なプロバイダー: `"openai"`, `"azure"`, `"google"` (Vertex AI), `"google_genai"` (Gemini API), `"anthropic"`, `"bedrock"`

In [ ]:
# === LLMプロバイダーの選択 ===
# 利用したいLLMプロバイダーを以下の変数で指定してください。
# "openai", "azure", "google" (Vertex AI), "google_genai" (Gemini API), "anthropic", "bedrock" のいずれかを選択できます。
LLM_PROVIDER = "openai"  # 例: OpenAI を利用する場合

### APIキー/環境変数の設定

以下のセルを実行する前に、選択したLLMプロバイダーに応じたAPIキーまたは環境変数を設定する必要があります。

**一般的な手順:**
1.  `.env.sample` ファイルをコピーして `.env` ファイルを作成します。
2.  `.env` ファイルを開き、選択したLLMプロバイダーに対応するAPIキーや必要な情報を記述します。
    *   **OpenAI:** `OPENAI_API_KEY`
    *   **Azure OpenAI:** `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_ENDPOINT`, `OPENAI_API_VERSION`, `AZURE_OPENAI_DEPLOYMENT_NAME`
    *   **Google (Vertex AI):** `GOOGLE_CLOUD_PROJECT_ID`, `GOOGLE_CLOUD_LOCATION` (Colab環境外で実行する場合、`GOOGLE_APPLICATION_CREDENTIALS` 環境変数の設定も必要になることがあります)
    *   **Google (Gemini API):** `GOOGLE_API_KEY`
    *   **Anthropic:** `ANTHROPIC_API_KEY`
    *   **AWS Bedrock:** `AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`, `AWS_REGION_NAME` (IAMロールを使用する場合は、これらのキー設定は不要な場合がありますが、リージョン名は必須です)
3.  ファイルを保存します。

**Google Colab を使用している場合:**
上記の `.env` ファイルを使用する代わりに、Colabのシークレットマネージャーに必要なキーを登録してください。
例えば、OpenAIを使用する場合は `OPENAI_API_KEY` という名前でシークレットを登録します。
Vertex AI を利用する場合は、Colab上での認証 (`google.colab.auth.authenticate_user()`) が実行されます。

このセルは、設定された情報に基づいて環境変数をロードし、LLMクライアントを初期化します。

In [ ]:
# === APIキー/環境変数の設定 ===
import os
from dotenv import load_dotenv

# .envファイルから環境変数を読み込む (存在する場合)
load_dotenv()

try:
    from google.colab import userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

# --- OpenAI ---
if LLM_PROVIDER == "openai":
    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY and IS_COLAB:
        OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY:
        raise ValueError("OpenAI APIキーが設定されていません。環境変数 OPENAI_API_KEY を設定するか、Colab環境の場合はシークレットに OPENAI_API_KEY を設定してください。")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# --- Azure OpenAI ---
elif LLM_PROVIDER == "azure":
    AZURE_OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
    AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
    OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
    AZURE_OPENAI_DEPLOYMENT_NAME = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if IS_COLAB:
        if not AZURE_OPENAI_API_KEY: AZURE_OPENAI_API_KEY = userdata.get("AZURE_OPENAI_API_KEY")
        if not AZURE_OPENAI_ENDPOINT: AZURE_OPENAI_ENDPOINT = userdata.get("AZURE_OPENAI_ENDPOINT")
        if not OPENAI_API_VERSION: OPENAI_API_VERSION = userdata.get("OPENAI_API_VERSION") # 例: "2023-07-01-preview"
        if not AZURE_OPENAI_DEPLOYMENT_NAME: AZURE_OPENAI_DEPLOYMENT_NAME = userdata.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if not AZURE_OPENAI_API_KEY: raise ValueError("Azure OpenAI APIキー (AZURE_OPENAI_API_KEY) が設定されていません。")
    if not AZURE_OPENAI_ENDPOINT: raise ValueError("Azure OpenAI エンドポイント (AZURE_OPENAI_ENDPOINT) が設定されていません。")
    if not OPENAI_API_VERSION: OPENAI_API_VERSION = "2023-07-01-preview" # デフォルトを設定することも可能
    if not AZURE_OPENAI_DEPLOYMENT_NAME: raise ValueError("Azure OpenAI デプロイメント名 (AZURE_OPENAI_DEPLOYMENT_NAME) が設定されていません。")

    os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
    os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
    os.environ["OPENAI_API_VERSION"] = OPENAI_API_VERSION

# --- Google Cloud Vertex AI (Gemini) ---
elif LLM_PROVIDER == "google":
    PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT_ID") # .env 用に修正
    LOCATION = os.environ.get("GOOGLE_CLOUD_LOCATION")

    if IS_COLAB:
        if not PROJECT_ID: PROJECT_ID = userdata.get("GOOGLE_CLOUD_PROJECT_ID")
        if not LOCATION: LOCATION = userdata.get("GOOGLE_CLOUD_LOCATION") # 例: "us-central1"
        from google.colab import auth as google_auth
        google_auth.authenticate_user() # Vertex AI を使う場合は Colab での認証を推奨
    else: # Colab外の場合、.envから読み込んだ値で環境変数を設定
        if PROJECT_ID: os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID # Vertex AI SDKが参照する標準的な環境変数名
        if LOCATION: os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION

    if not PROJECT_ID: raise ValueError("Google Cloud Project ID が設定されていません。環境変数 GOOGLE_CLOUD_PROJECT_ID を設定するか、Colab環境の場合はシークレットに GOOGLE_CLOUD_PROJECT_ID を設定してください。")
    if not LOCATION: LOCATION = "us-central1" # デフォルトロケーション

# --- Google Gemini API (langchain-google-genai) ---
elif LLM_PROVIDER == "google_genai":
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY and IS_COLAB:
        GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        raise ValueError("Google APIキーが設定されていません。環境変数 GOOGLE_API_KEY を設定するか、Colab環境の場合はシークレットに GOOGLE_API_KEY を設定してください。")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# --- Anthropic (Claude) ---
elif LLM_PROVIDER == "anthropic":
    ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY and IS_COLAB:
        ANTHROPIC_API_KEY = userdata.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY:
        raise ValueError("Anthropic APIキーが設定されていません。環境変数 ANTHROPIC_API_KEY を設定するか、Colab環境の場合はシークレットに ANTHROPIC_API_KEY を設定してください。")
    os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

# --- Amazon Bedrock (Claude) ---
elif LLM_PROVIDER == "bedrock":
    AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
    AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
    AWS_REGION_NAME = os.environ.get("AWS_REGION_NAME")

    if IS_COLAB: 
        if not AWS_ACCESS_KEY_ID: AWS_ACCESS_KEY_ID = userdata.get("AWS_ACCESS_KEY_ID")
        if not AWS_SECRET_ACCESS_KEY: AWS_SECRET_ACCESS_KEY = userdata.get("AWS_SECRET_ACCESS_KEY")
        if not AWS_REGION_NAME: AWS_REGION_NAME = userdata.get("AWS_REGION_NAME")

    if not AWS_REGION_NAME:
         raise ValueError("AWSリージョン名 (AWS_REGION_NAME) が設定されていません。Bedrock利用にはリージョン指定が必要です。")

    # 環境変数に設定 (boto3がこれらを自動で読み込む)
    if AWS_ACCESS_KEY_ID: os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
    if AWS_SECRET_ACCESS_KEY: os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
    os.environ["AWS_DEFAULT_REGION"] = AWS_REGION_NAME # boto3が参照する標準的なリージョン環境変数名
    os.environ["AWS_REGION"] = AWS_REGION_NAME # いくつかのライブラリはこちらを参照することもある

print(f"APIキー/環境変数の設定完了 (プロバイダー: {LLM_PROVIDER})")

### LLMクライアントの初期化

このセルは、上で選択・設定したLLMプロバイダーに基づいて、対応するLLMクライアントを初期化します。

In [ ]:
# === LLMクライアントの動的初期化 ===
llm = None

if LLM_PROVIDER == "openai":
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
elif LLM_PROVIDER == "azure":
    from langchain_openai import AzureChatOpenAI
    llm = AzureChatOpenAI(
        azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME"), # 環境変数から取得
        openai_api_version=os.environ.get("OPENAI_API_VERSION"), # 環境変数から取得
        temperature=0,
    )
elif LLM_PROVIDER == "google":
    from langchain_google_vertexai import ChatVertexAI
    # PROJECT_ID, LOCATION は前のセルで環境変数に設定済みか、Colabの場合は直接利用
    llm = ChatVertexAI(model_name="gemini-1.5-flash-001", temperature=0, project=os.environ.get("GOOGLE_CLOUD_PROJECT"), location=os.environ.get("GOOGLE_CLOUD_LOCATION"))
elif LLM_PROVIDER == "google_genai":
    from langchain_google_genai import ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0, convert_system_message_to_human=True)
elif LLM_PROVIDER == "anthropic":
    from langchain_anthropic import ChatAnthropic
    llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)
elif LLM_PROVIDER == "bedrock":
    from langchain_aws import ChatBedrock # langchain_community.chat_models から langchain_aws に変更の可能性あり
    # AWS_REGION_NAME は前のセルで環境変数 AWS_DEFAULT_REGION に設定済み
    llm = ChatBedrock( # BedrockChat ではなく ChatBedrock が一般的
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        # region_name=os.environ.get("AWS_DEFAULT_REGION"), # 通常、boto3が環境変数から自動で読み込む
        model_kwargs={"temperature": 0},
    )
else:
    raise ValueError(
        f"Unsupported LLM_PROVIDER: {LLM_PROVIDER}. "
        "Please choose from 'openai', 'azure', 'google', 'google_genai', 'anthropic', or 'bedrock'."
    )

print(f"LLM Provider: {LLM_PROVIDER}")
if llm:
    print(f"LLM Client Type: {type(llm)}")
    # モデル名取得の試行を汎用的に
    model_attr = getattr(llm, 'model', None) or \ 
                 getattr(llm, 'model_name', None) or \ 
                 getattr(llm, 'model_id', None) or \ 
                 (hasattr(llm, 'llm') and getattr(llm.llm, 'model', None)) # 一部のLLMクライアントのネスト構造に対応
    if hasattr(llm, 'azure_deployment') and not model_attr: # Azure特有の属性
        model_attr = llm.azure_deployment
        
    if model_attr:
        print(f"LLM Model: {model_attr}")
    else:
        print("LLM Model: (Could not determine model name from client attributes)")


---

この章では、ToolNodeの活用やPlan-and-Execute型エージェントの構築など、ツール連携を含むシングルエージェントの作成方法を学びます。

### ■ 問題001: 基本的なツールの実装とToolNodeの使用

LangGraphで外部ツールを利用するための第一歩として、簡単な計算ツールを作成し、それを`ToolNode`を通じてエージェントに組み込む方法を学びます。LLMがユーザーの指示に応じてこのツールを呼び出し、その結果を返す基本的なエージェントを構築します。

In [ ]:
# 解答欄001
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

# --- 状態定義 (AgentState) ---
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

# --- ツール定義 ---
@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers a and b."""
    print(f"Calling multiply tool with: {a}, {b}")
    return a * b

tools = [multiply]

# --- ノード定義 ---

# LLMを呼び出してツールの使用を決定するノード
def agent_node(state: AgentState):
    print("agent_node: Calling LLM to decide on tool use...")
    # LLMにツールをバインドして呼び出し (llmはノートブック冒頭で初期化済み)
    # ヒント: llm.bind_tools(tools) を使ってLLMにツールを使えるように指示します
    llm_with_tools = ____.____(____) 
    response = llm_with_tools.invoke(state["messages"])
    print(f"agent_node: LLM response: {response}")
    # AIMessageを返す（ToolCallを含む場合も、含まない場合もある）
    return {"messages": [response]}

# ToolNodeのインスタンス化
# ヒント: ToolNode に tools リストを渡します
tool_node = ____(____)

# --- グラフ構築 ---
workflow = StateGraph(AgentState)

# ノードの追加
workflow.add_node("agent", agent_node)
workflow.add_node("tools", ____) # tool_node を追加

# エントリポイントの設定
workflow.set_entry_point("agent")

# エッジの定義
# agentノードの後、LLMがツールコールを生成したかどうかで分岐する
def should_continue(state: AgentState):
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        print("should_continue: Routing to tools")
        return "tools"  # ツールコールがあればToolNodeへ
    else:
        print("should_continue: Routing to END")
        return END      # なければ終了

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",
        END: END
    }
)

# ToolNodeからagentノードへ戻るエッジ (ツール実行結果をLLMに渡して最終応答を生成させるため)
workflow.add_edge("tools", "agent")

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- ツール利用のテスト (掛け算) ---")
inputs1 = {"messages": [HumanMessage(content="5と6を掛けてください。")]}
for s in app.stream(inputs1, {"recursion_limit": 5}): # ループの可能性があるのでrecursion_limitを設定
    print(s)
final_state1 = app.invoke(inputs1, {"recursion_limit": 5})
print(f"Final State 1: {final_state1['messages'][-1].content}")

print("\n--- ツールを利用しない通常の会話テスト ---")
inputs2 = {"messages": [HumanMessage(content="こんにちは！調子はどうですか？")]}
for s in app.stream(inputs2, {"recursion_limit": 5}):
    print(s)
final_state2 = app.invoke(inputs2, {"recursion_limit": 5})
print(f"Final State 2: {final_state2['messages'][-1].content}")

<details><summary>解答001</summary>

```python
# 解答001
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

# --- 状態定義 (AgentState) ---
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

# --- ツール定義 ---
@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers a and b."""
    print(f"Calling multiply tool with: {a}, {b}")
    return a * b

tools = [multiply]

# --- ノード定義 ---

# LLMを呼び出してツールの使用を決定するノード
def agent_node(state: AgentState):
    print("agent_node: Calling LLM to decide on tool use...")
    # LLMにツールをバインドして呼び出し (llmはノートブック冒頭で初期化済み)
    llm_with_tools = llm.bind_tools(tools)
    response = llm_with_tools.invoke(state["messages"])
    print(f"agent_node: LLM response: {response}")
    # AIMessageを返す（ToolCallを含む場合も、含まない場合もある）
    return {"messages": [response]}

# ToolNodeのインスタンス化
tool_node = ToolNode(tools)

# --- グラフ構築 ---
workflow = StateGraph(AgentState)

# ノードの追加
workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node) # tool_node を追加

# エントリポイントの設定
workflow.set_entry_point("agent")

# エッジの定義
# agentノードの後、LLMがツールコールを生成したかどうかで分岐する
def should_continue(state: AgentState):
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        print("should_continue: Routing to tools")
        return "tools"  # ツールコールがあればToolNodeへ
    else:
        print("should_continue: Routing to END")
        return END      # なければ終了

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",
        END: END
    }
)

# ToolNodeからagentノードへ戻るエッジ (ツール実行結果をLLMに渡して最終応答を生成させるため)
workflow.add_edge("tools", "agent")

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- ツール利用のテスト (掛け算) ---")
inputs1 = {"messages": [HumanMessage(content="5と6を掛けてください。")]}
for s in app.stream(inputs1, {"recursion_limit": 5}): # ループの可能性があるのでrecursion_limitを設定
    print(s)
final_state1 = app.invoke(inputs1, {"recursion_limit": 5})
print(f"Final State 1: {final_state1['messages'][-1].content}")

print("\n--- ツールを利用しない通常の会話テスト ---")
inputs2 = {"messages": [HumanMessage(content="こんにちは！調子はどうですか？")]}
for s in app.stream(inputs2, {"recursion_limit": 5}):
    print(s)
final_state2 = app.invoke(inputs2, {"recursion_limit": 5})
print(f"Final State 2: {final_state2['messages'][-1].content}")
```
</details>

<details><summary>解説001</summary>

#### この問題のポイント
*   **学習内容:**
    *   LangChainの`@tool`デコレータを使ってPython関数をツールとして定義する方法。
    *   `llm.bind_tools(tools)`を使って、LLMが利用可能なツールを認識できるようにする方法。
    *   `ToolNode`を作成し、グラフに組み込むことで、LLMが生成したツールコールリクエスト(`AIMessage.tool_calls`)を実際のツール実行に繋げる方法。
    *   条件付きエッジ (`add_conditional_edges`) を使用して、LLMの応答にツールコールが含まれているかどうかに基づいて処理フローを分岐させる方法。
    *   ツール実行後、その結果 (`ToolMessage`) を含む更新された状態で再度LLMノードを呼び出し、最終的な応答を生成させる基本的なエージェントループの構築。
*   **コード解説:**
    *   `AgentState`: 状態定義はシンプルに`messages`のみを保持します。`add_messages`により、新しいメッセージはリストの末尾に追加されます。
    *   `@tool def multiply(...)`: `multiply`関数をLangChainツールとして定義します。型アノテーションとdocstringは、LLMがツールを正しく理解し使用するために重要です。
    *   `tools = [multiply]`: 利用可能なツールのリストを作成します。
    *   `agent_node`: このノードがエージェントの頭脳部分です。
        *   `llm.bind_tools(tools)`: ノートブック冒頭で初期化された`llm`オブジェクトに`tools`リストをバインドします。これにより、LLMはこれらのツールの存在と使い方（docstringや型情報から）を認識し、必要に応じて`tool_calls`属性を持つ`AIMessage`を生成できるようになります。
        *   `llm_with_tools.invoke(state["messages"])`: 現在のメッセージ履歴をLLMに渡し、応答を生成させます。この応答は、通常のテキスト応答であることも、ツール使用を指示する`tool_calls`を含むこともあります。
    *   `tool_node = ToolNode(tools)`: `ToolNode`はLangGraphが提供する便利なノードで、`AIMessage`に含まれる`tool_calls`を検出し、対応するツールを実行し、その結果を`ToolMessage`として返します。
    *   `workflow.add_conditional_edges("agent", should_continue, ...)`: `agent_node`の実行後、`should_continue`関数が呼び出されます。
        *   `should_continue`: 最新のメッセージ (`state["messages"][-1]`) が`tool_calls`属性を持ち、それが空でない場合（つまりLLMがツール使用を指示した場合）は`"tools"`（`ToolNode`の名前）を返します。そうでなければ`END`を返し、グラフの実行を終了します。
    *   `workflow.add_edge("tools", "agent")`: `ToolNode`がツールを実行した後、その結果（`ToolMessage`として`messages`に追加される）を持って再度`agent_node`に戻ります。これにより、LLMはツールの実行結果を踏まえて最終的なユーザーへの応答を生成できます。
    *   `app.stream(inputs, {"recursion_limit": 5})`: `agent` -> `tools` -> `agent` というループが発生する可能性があるため、無限ループを防ぐために`recursion_limit`を設定しています。この制限は、グラフが同じノードシーケンスを何回繰り返すことができるかを制御します。

この問題を通じて、LangGraphにおけるツールの基本的な組み込み方と、LLMを中心としたシンプルなエージェントの動作原理を理解できます。
---
</details>

In [ ]:
# 今後の実装に向けた空のコードセル

### ■ 問題002: ReAct風エージェントロジック - LLMによるツール使用判断と応答選択

問題001で構築したツール利用の基本形を発展させ、LLMがより明示的に「ツールを使うべきか、それとも直接応答すべきか」を判断するロジックを組み込みます。この問題では、LLMの判断に基づいてツール実行へ進むか、または直接ユーザーに応答を返して処理を終了するかを制御する、よりReAct（Reason + Act）に近い思考プロセスを持つエージェントの基礎を構築します。

In [ ]:
# 解答欄002
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

# --- 状態定義 (AgentState) ---
# 問題001と同じAgentStateを使用します
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

# --- ツール定義 ---
# 問題001と同じツールを使用します
@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers 'a' and 'b'."""
    print(f"Calling multiply tool with: {a}, {b}")
    return a * b

tools = [multiply]
llm_with_tools = llm.bind_tools(tools) # llmはノートブック冒頭で初期化済み

# --- ノード定義 ---

# エージェントノード: LLMがツール呼び出しを行うか、直接応答するかを決定
def agent_decision_node(state: AgentState):
    print("agent_decision_node: LLM making decision...")
    # ヒント: 最新のメッセージリストでLLMを呼び出します。
    # llm_with_tools は既にツールがバインドされているため、LLMはツール使用を適切に判断できます。
    response = ____.invoke(____["messages"])
    print(f"agent_decision_node: LLM response: {response}")
    # AIMessageを返し、ツールコールが含まれていればToolNodeへ、そうでなければENDへ遷移するための判断材料となります。
    return {"messages": [response]}

# ToolNodeのインスタンス化 (問題001と同様)
tool_node = ToolNode(tools)

# --- グラフ構築 ---
workflow = StateGraph(AgentState)

workflow.add_node("agent", agent_decision_node)
workflow.add_node("tools", tool_node)

workflow.set_entry_point("agent")

# 条件付きエッジ: LLMの応答に基づいてルーティング
def route_after_decision(state: AgentState):
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and ____.____:
        print("route_after_decision: Routing to tools")
        return "tools"  # ツールコールがあればToolNodeへ
    else:
        print("route_after_decision: Routing to END")
        return ____      # なければ終了 (直接応答)

workflow.add_conditional_edges(
    "agent",
    route_after_decision,
    {
        "tools": "tools",
        END: END
    }
)

# ツール実行後、再度エージェントノードに戻り、ツール結果を踏まえた最終応答を生成
workflow.add_edge("tools", "agent")

app_react_style = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- ReAct風エージェント: ツール利用のテスト (掛け算) ---")
inputs_react1 = {"messages": [HumanMessage(content="12と8を掛けるといくつになりますか？")]}
for s in app_react_style.stream(inputs_react1, {"recursion_limit": 5}):
    print(s)
final_state_react1 = app_react_style.invoke(inputs_react1, {"recursion_limit": 5})
print(f"Final State ReAct 1: {final_state_react1['messages'][-1].content}")

print("\n--- ReAct風エージェント: ツールを利用しない直接応答テスト ---")
inputs_react2 = {"messages": [HumanMessage(content="今日の東京の天気は？ (ツール外の質問)")]}
for s in app_react_style.stream(inputs_react2, {"recursion_limit": 5}):
    print(s)
final_state_react2 = app_react_style.invoke(inputs_react2, {"recursion_limit": 5})
print(f"Final State ReAct 2: {final_state_react2['messages'][-1].content}")


<details><summary>解答002</summary>

```python
# 解答002
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

# --- 状態定義 (AgentState) ---
# 問題001と同じAgentStateを使用します
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

# --- ツール定義 ---
# 問題001と同じツールを使用します
@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers 'a' and 'b'."""
    print(f"Calling multiply tool with: {a}, {b}")
    return a * b

tools = [multiply]
llm_with_tools = llm.bind_tools(tools) # llmはノートブック冒頭で初期化済み

# --- ノード定義 ---

# エージェントノード: LLMがツール呼び出しを行うか、直接応答するかを決定
def agent_decision_node(state: AgentState):
    print("agent_decision_node: LLM making decision...")
    # ヒント: 最新のメッセージリストでLLMを呼び出します。
    # llm_with_tools は既にツールがバインドされているため、LLMはツール使用を適切に判断できます。
    response = llm_with_tools.invoke(state["messages"])
    print(f"agent_decision_node: LLM response: {response}")
    # AIMessageを返し、ツールコールが含まれていればToolNodeへ、そうでなければENDへ遷移するための判断材料となります。
    return {"messages": [response]}

# ToolNodeのインスタンス化 (問題001と同様)
tool_node = ToolNode(tools)

# --- グラフ構築 ---
workflow = StateGraph(AgentState)

workflow.add_node("agent", agent_decision_node)
workflow.add_node("tools", tool_node)

workflow.set_entry_point("agent")

# 条件付きエッジ: LLMの応答に基づいてルーティング
def route_after_decision(state: AgentState):
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        print("route_after_decision: Routing to tools")
        return "tools"  # ツールコールがあればToolNodeへ
    else:
        print("route_after_decision: Routing to END")
        return END      # なければ終了 (直接応答)

workflow.add_conditional_edges(
    "agent",
    route_after_decision,
    {
        "tools": "tools",
        END: END
    }
)

# ツール実行後、再度エージェントノードに戻り、ツール結果を踏まえた最終応答を生成
workflow.add_edge("tools", "agent") # ツール実行結果を agent に渡して再度判断させる

app_react_style = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- ReAct風エージェント: ツール利用のテスト (掛け算) ---")
inputs_react1 = {"messages": [HumanMessage(content="12と8を掛けるといくつになりますか？")]}
for s in app_react_style.stream(inputs_react1, {"recursion_limit": 5}):
    print(s)
final_state_react1 = app_react_style.invoke(inputs_react1, {"recursion_limit": 5})
print(f"Final State ReAct 1: {final_state_react1['messages'][-1].content}")

print("\n--- ReAct風エージェント: ツールを利用しない直接応答テスト ---")
inputs_react2 = {"messages": [HumanMessage(content="今日の東京の天気は？ (ツール外の質問)")]}
for s in app_react_style.stream(inputs_react2, {"recursion_limit": 5}):
    print(s)
final_state_react2 = app_react_style.invoke(inputs_react2, {"recursion_limit": 5})
print(f"Final State ReAct 2: {final_state_react2['messages'][-1].content}")
```
</details>

<details><summary>解説002</summary>

#### この問題のポイント
*   **学習内容:**
    *   LLMがツール使用の判断と直接応答の判断をどのように行うか、その基本的な流れを理解する。
    *   `llm.bind_tools()` を使用することで、LLMが自律的にツールコールを生成するか、通常のテキスト応答を返すかを選択できるようになることの再確認。
    *   条件付きエッジ `add_conditional_edges` を用いて、LLMの判断（`tool_calls` の有無）に応じて処理フローを分岐させる方法。
    *   ツール実行後、その結果（`ToolMessage`）をエージェント（LLMノード）にフィードバックし、最終的な応答を生成させるループ構造の重要性。
*   **コード解説:**
    *   `agent_decision_node`: このノードは問題001の`agent_node`とほぼ同じ役割を担います。`llm.bind_tools(tools)`によってツールがLLMにバインドされているため、LLMは入力メッセージに基づいてツールを使用すべきか、直接応答すべきかを判断します。その結果、`AIMessage`が返され、これに`tool_calls`が含まれていればツール実行へ、含まれていなければ（つまりLLMが直接応答を生成したと判断されれば）処理終了へと分岐します。
    *   `route_after_decision`: この条件分岐関数は問題001の`should_continue`と同じです。最新の`AIMessage`に`tool_calls`が含まれていれば`"tools"`（ToolNodeへ）を、そうでなければ`END`（処理終了）を返します。
    *   `workflow.add_edge("tools", "agent")`: このエッジは非常に重要です。ToolNodeがツールを実行した後、その結果（`ToolMessage`として`messages`状態に追加される）を伴って、再度`agent_decision_node`に戻ります。これにより、LLMはツールの実行結果を考慮して、ユーザーへの最終的な応答を生成することができます。例えば、「5と6を掛けて」というリクエストに対し、LLMが`multiply(a=5, b=6)`というツールコールを生成 → ToolNodeが実行して`30`という結果を返す → この結果が`ToolMessage`として`agent_decision_node`に渡され、LLMが「5と6を掛けた結果は30です。」のような最終応答を生成します。
    *   **ReAct風の思考プロセス:** この構造は、ReAct（Reason + Act）パラダイムの基本的な要素を示しています。
        1.  **Reason (推論):** `agent_decision_node`でLLMが現在の状態（メッセージ履歴）に基づいて次に何をすべきか（ツールを使うか、応答するか）を推論します。
        2.  **Act (行動):** 推論の結果、ツール使用が選択されれば`ToolNode`が行動（ツール実行）し、直接応答が選択されればグラフが終了（応答完了）します。ツール実行後は、その結果を持って再度Reasonのステップに戻ります。
    *   **LLMの能力への依存:** 近年の一部の高性能なLLM（例: GPT-4o, Gemini 1.5 Proなど）は、`bind_tools`でツールを渡すだけで、プロンプトで明示的に「ツールを使うか考えて」と指示しなくても、適切にツールコールを生成したり直接応答したりする能力が高まっています。この問題の構造は、そのようなLLMの能力を前提として、グラフの制御フローに焦点を当てています。

この問題を通じて、LLMの判断に基づいて動的に処理フローを分岐させ、ツール利用と応答生成を組み合わせるエージェントの基本的な形を学ぶことができます。これはより複雑な自律型エージェントを構築するための重要なステップです。
---
</details>

In [ ]:
# 今後の実装に向けた空のコードセル

### ■ 問題003: ツール呼び出しにおけるエージェント状態管理の理解

これまでの問題で、エージェントがツールを呼び出し、その結果を受け取る流れを構築しました。この問題では、特にツール実行結果である`ToolMessage`がエージェントの状態(`AgentState`)にどのように追加され、その後のLLMの応答生成にどう影響を与えるかに焦点を当てます。`ToolNode`が返す`ToolMessage`がメッセージ履歴に正しく組み込まれ、LLMがそれを参照して最終的な回答を形成するプロセスを明確に理解しましょう。

In [ ]:
# 解答欄003
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

# --- 状態定義 (AgentState) ---
# これまでの問題と同じAgentStateを使用
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

# --- ツール定義 ---
# より明確なツール結果の確認のため、少しメッセージを変えたツールを定義
@tool
def get_capital_city(country_name: str) -> str:
    """Provides the capital city of a given country. 
    Only knows about Japan, France, and Canada."""
    print(f"get_capital_city tool called with: {country_name}")
    capitals = {
        "japan": "Tokyo",
        "france": "Paris",
        "canada": "Ottawa"
    }
    return capitals.get(country_name.lower(), f"Sorry, I don't know the capital of {country_name}.")

tools_city = [get_capital_city]
llm_with_city_tool = llm.bind_tools(tools_city) # llmはノートブック冒頭で初期化済み

# --- ノード定義 ---
def agent_node_state_mgmt(state: AgentState):
    print(f"\nagent_node_state_mgmt: Current messages in state: {[msg.pretty_repr() for msg in state['messages']]}")
    # 最新のメッセージリスト（ToolMessageが含まれている可能性がある）でLLMを呼び出し
    # ヒント: state['messages'] をそのまま llm_with_city_tool に渡します。
    # ToolMessage が履歴にあれば、LLMはそれを考慮して応答を生成します。
    response = ____.invoke(____)
    print(f"agent_node_state_mgmt: LLM response: {response.pretty_repr()}")
    return {"messages": [response]}

tool_node_city = ToolNode(tools_city)

# --- グラフ構築 ---
workflow_state_mgmt = StateGraph(AgentState)

workflow_state_mgmt.add_node("agent_sm", agent_node_state_mgmt)
workflow_state_mgmt.add_node("tools_sm", tool_node_city)

workflow_state_mgmt.set_entry_point("agent_sm")

def route_after_agent_sm(state: AgentState):
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        print("route_after_agent_sm: Routing to tools_sm")
        return "tools_sm"
    else:
        print("route_after_agent_sm: Routing to END")
        return END

workflow_state_mgmt.add_conditional_edges(
    "agent_sm",
    route_after_agent_sm,
    {
        "tools_sm": "tools_sm",
        END: END
    }
)

# ToolNodeからagentノードへ戻るエッジは必須
workflow_state_mgmt.____("tools_sm", "agent_sm")

app_state_mgmt = workflow_state_mgmt.compile()

# --- グラフの実行と結果表示 ---
print("\n--- 状態管理テスト: 日本の首都を尋ねる ---")
inputs_sm1 = {"messages": [HumanMessage(content="日本の首都はどこですか？")]}
for i, s in enumerate(app_state_mgmt.stream(inputs_sm1, {"recursion_limit": 5})):
    print(f"Stream output {i}: {s}")
    print("----")
final_state_sm1 = app_state_mgmt.invoke(inputs_sm1, {"recursion_limit": 5})
print(f"Final State SM1: {final_state_sm1['messages'][-1].content}")

print("\n--- 状態管理テスト: ドイツの首都を尋ねる (ツールは知らない) ---")
inputs_sm2 = {"messages": [HumanMessage(content="ドイツの首都はどこですか？")]}
for i, s in enumerate(app_state_mgmt.stream(inputs_sm2, {"recursion_limit": 5})):
    print(f"Stream output {i}: {s}")
    print("----")
final_state_sm2 = app_state_mgmt.invoke(inputs_sm2, {"recursion_limit": 5})
print(f"Final State SM2: {final_state_sm2['messages'][-1].content}")

<details><summary>解答003</summary>

```python
# 解答003
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

# --- 状態定義 (AgentState) ---
# これまでの問題と同じAgentStateを使用
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

# --- ツール定義 ---
# より明確なツール結果の確認のため、少しメッセージを変えたツールを定義
@tool
def get_capital_city(country_name: str) -> str:
    """Provides the capital city of a given country. 
    Only knows about Japan, France, and Canada."""
    print(f"get_capital_city tool called with: {country_name}")
    capitals = {
        "japan": "Tokyo",
        "france": "Paris",
        "canada": "Ottawa"
    }
    return capitals.get(country_name.lower(), f"Sorry, I don't know the capital of {country_name}.")

tools_city = [get_capital_city]
llm_with_city_tool = llm.bind_tools(tools_city) # llmはノートブック冒頭で初期化済み

# --- ノード定義 ---
def agent_node_state_mgmt(state: AgentState):
    print(f"\nagent_node_state_mgmt: Current messages in state: {[msg.pretty_repr() for msg in state['messages']]}")
    # 最新のメッセージリスト（ToolMessageが含まれている可能性がある）でLLMを呼び出し
    # ヒント: state['messages'] をそのまま llm_with_city_tool に渡します。
    # ToolMessage が履歴にあれば、LLMはそれを考慮して応答を生成します。
    response = llm_with_city_tool.invoke(state['messages'])
    print(f"agent_node_state_mgmt: LLM response: {response.pretty_repr()}")
    return {"messages": [response]}

tool_node_city = ToolNode(tools_city)

# --- グラフ構築 ---
workflow_state_mgmt = StateGraph(AgentState)

workflow_state_mgmt.add_node("agent_sm", agent_node_state_mgmt)
workflow_state_mgmt.add_node("tools_sm", tool_node_city)

workflow_state_mgmt.set_entry_point("agent_sm")

def route_after_agent_sm(state: AgentState):
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        print("route_after_agent_sm: Routing to tools_sm")
        return "tools_sm"
    else:
        print("route_after_agent_sm: Routing to END")
        return END

workflow_state_mgmt.add_conditional_edges(
    "agent_sm",
    route_after_agent_sm,
    {
        "tools_sm": "tools_sm",
        END: END
    }
)

# ToolNodeからagentノードへ戻るエッジは必須
workflow_state_mgmt.add_edge("tools_sm", "agent_sm")

app_state_mgmt = workflow_state_mgmt.compile()

# --- グラフの実行と結果表示 ---
print("\n--- 状態管理テスト: 日本の首都を尋ねる ---")
inputs_sm1 = {"messages": [HumanMessage(content="日本の首都はどこですか？")]}
for i, s in enumerate(app_state_mgmt.stream(inputs_sm1, {"recursion_limit": 5})):
    print(f"Stream output {i}: {s}")
    # 各ステップでのmessagesの状態を確認するために詳細出力（デバッグ用）
    # current_messages = s.get(list(s.keys())[0]).get('messages')
    # if current_messages:
    #     print(f"  Messages @ step {i}: {[msg.pretty_repr() for msg in current_messages]}")
    print("----")
final_state_sm1 = app_state_mgmt.invoke(inputs_sm1, {"recursion_limit": 5})
print(f"Final State SM1: {final_state_sm1['messages'][-1].content}")

print("\n--- 状態管理テスト: ドイツの首都を尋ねる (ツールは知らない) ---")
inputs_sm2 = {"messages": [HumanMessage(content="ドイツの首都はどこですか？")]}
for i, s in enumerate(app_state_mgmt.stream(inputs_sm2, {"recursion_limit": 5})):
    print(f"Stream output {i}: {s}")
    print("----")
final_state_sm2 = app_state_mgmt.invoke(inputs_sm2, {"recursion_limit": 5})
print(f"Final State SM2: {final_state_sm2['messages'][-1].content}")
```
</details>

<details><summary>解説003</summary>

#### この問題のポイント
*   **学習内容:**
    *   `ToolMessage`がエージェントの状態(`AgentState`の`messages`リスト)にどのように追加され、蓄積されていくかの具体的な確認。
    *   LLMノード（ここでは`agent_node_state_mgmt`）が、ツール実行結果である`ToolMessage`を含むメッセージ履歴全体を受け取り、それを踏まえて次のアクション（最終応答の生成や、さらなるツール呼び出しなど）を決定する流れの理解。
    *   `add_messages`修飾子と`ToolNode`の組み合わせが、ツール呼び出しサイクルにおける状態管理をいかに自動化し、簡潔にしているかの確認。
*   **コード解説:**
    *   **新しいツール `get_capital_city`**: この問題では、より具体的なツール「首都検索ツール」を導入しました。これにより、ツールコールとツールからの戻り値（`ToolMessage`の内容）が明確になり、状態変化を追いやすくなります。
    *   **`agent_node_state_mgmt`内のログ出力**: `print(f"\nagent_node_state_mgmt: Current messages in state: {[msg.pretty_repr() for msg in state['messages']]}")`という行を追加しました。これにより、エージェントノードが呼び出されるたびに、現在の`messages`状態がコンソールに出力されます。ここには、ユーザーの最初の質問(`HumanMessage`)、LLMによるツール呼び出し指示(`AIMessage` with `tool_calls`)、そしてツールの実行結果(`ToolMessage`)が順番に蓄積されていく様子が確認できます。
    *   **LLMへの入力**: `response = llm_with_city_tool.invoke(state['messages'])`の部分では、更新された`state['messages']`全体（先行する`ToolMessage`も含む）がLLMに渡されます。LLMは`ToolMessage`の内容（例: `ToolMessage(content='Tokyo', tool_call_id='...')`）を参照し、「日本の首都は東京です。」のような最終応答を生成します。
    *   **グラフ構造の再確認**: グラフの構造（`agent` -> `tools` -> `agent`のループ）は問題002と基本的に同じです。重要なのは、このループの中で`messages`状態がどのように更新され、各コンポーネント（特にLLMノード）がその状態をどのように利用しているかを意識することです。
    *   **`ToolNode`の役割**: `ToolNode`は、`AIMessage`内の`tool_calls`に基づいて指定されたツールを実行し、その結果を`ToolMessage`として自動的に生成してくれます。この`ToolMessage`には、元の`tool_call_id`が含まれるため、LLMはどのツール呼び出しに対する結果なのかを正しく関連付けることができます。そして`add_messages`がこの`ToolMessage`を`messages`リストに追加します。

#### 実行時の確認ポイント
コードを実行すると、`agent_node_state_mgmt`内のprint文によって、メッセージの変遷が詳細に出力されます。
1.  最初の`agent_node_state_mgmt`呼び出し時: `messages`には`HumanMessage`のみが含まれます。
2.  LLMがツールコールを決定すると、`AIMessage(..., tool_calls=[...])`が`messages`に追加されます。
3.  `tool_node_city`が実行されると、その結果として`ToolMessage(content='Tokyo', ...)`が`messages`に追加されます。
4.  再度`agent_node_state_mgmt`が呼び出される時: `messages`には`HumanMessage`, `AIMessage(tool_calls)`, `ToolMessage`の3つが含まれています。LLMはこの完全な履歴を見て、最終的な応答を生成します。

この一連の流れを追うことで、LangGraphにおけるツール連携時の状態管理の仕組み、特に`messages`リストの役割と`ToolMessage`の重要性が明確に理解できます。
---
</details>

In [ ]:
# 今後の実装に向けた空のコードセル

### ■ 問題004: シンプルなPlan-and-Executeエージェントの構築

より複雑なタスクに対応するために、Plan-and-Execute（計画と実行）という考え方を導入します。このエージェントは、まずユーザーのリクエストに基づいて行動計画を立て（Plan）、その後、その計画に従ってステップを実行します（Execute）。
この問題では、まず簡単な計画を生成する「プランナー」ノードと、その計画の最初のステップを実行（または実行を試みる）「エグゼキューター」ノードを持つエージェントを構築します。
状態には新たに `input` (ユーザーの初期リクエスト) と `plan` (文字列のリストとしての計画ステップ) を追加します。

In [ ]:
# 解答欄004
from typing import TypedDict, Annotated, Sequence, List, Optional
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

# --- 状態定義 (PlanAndExecuteState) ---
class PlanAndExecuteState(TypedDict):
    input: str
    plan: List[str]
    # 実行されたステップの結果や途中の思考を保持（オプション）
    past_steps: Annotated[List[tuple[str, str]], lambda x, y: x + y]
    # messages は引き続きツール利用時のコミュニケーションで使用
    messages: Annotated[Sequence[BaseMessage], add_messages]
    # 現在の計画ステップに対する最終結果（ユーザーへの応答）
    response: Optional[str]

# --- ツール定義 (問題003のget_capital_cityを再利用) ---
@tool
def get_capital_city(country_name: str) -> str:
    """Provides the capital city of a given country. 
    Only knows about Japan, France, and Canada."""
    print(f"get_capital_city tool called with: {country_name}")
    capitals = {
        "japan": "Tokyo",
        "france": "Paris",
        "canada": "Ottawa"
    }
    return capitals.get(country_name.lower(), f"Sorry, I don't know the capital of {country_name}.")

tools_pne = [get_capital_city]
llm_with_pne_tools = llm.bind_tools(tools_pne)

# --- プランナーノード ---
# LLMの応答から計画をパースするためのPydanticモデル
class Plan(BaseModel):
    steps: List[str] = Field(description="ユーザーのリクエストを達成するためのステップバイステップの計画")

def planner_node(state: PlanAndExecuteState):
    print(f"\nplanner_node: Generating plan for input: {state['input']}")
    # 構造化出力が可能なLLMを使用 (例: OpenAI Functions, Gemini Function Calling)
    # ヒント: llm.with_structured_output(Plan) を使って、LLMがPlanモデルに従った形式で出力するようにします。
    structured_llm_planner = ____.____(____) 
    plan_result = structured_llm_planner.invoke(f"ユーザーリクエスト: {state['input']}\n上記のユーザーリクエストを達成するための計画をステップのリストとして生成してください。各ステップは具体的で実行可能なアクションであるべきです。例えば、「日本の首都を調べる」など。")
    
    print(f"planner_node: Generated plan: {plan_result.steps}")
    return {"plan": plan_result.steps, "past_steps": []} # planを更新し、past_stepsを初期化

# --- エグゼキューターノード ---
# このエグゼキューターは、計画の最初の未実行ステップを取り出し、
# それを実行しようとします（ツールを使うか、LLMで直接応答）。
def executor_node(state: PlanAndExecuteState):
    plan = state["plan"]
    past_steps = state["past_steps"]
    
    if not plan: # 計画がない、または完了した場合
        print("executor_node: No plan or plan complete.")
        # ここではmessagesの最後のAIメッセージを最終応答とするか、別途集約ノードを設ける
        # 簡単のため、最後のメッセージが最終応答と仮定 (より洗練された方法も可能)
        final_ai_message = next((m for m in reversed(state['messages']) if isinstance(m, AIMessage) and not m.tool_calls), None)
        return {"response": final_ai_message.content if final_ai_message else "計画を実行しましたが、明確な最終応答がありませんでした。"}

    current_step_description = plan[0] # 最初のステップを取得
    print(f"\nexecutor_node: Executing step: {current_step_description}")

    # LLMに現在のステップの実行を依頼（ツール使用も考慮）
    # 注意: ここでは簡単のため、executor_nodeが直接LLMを呼び出し、ツール使用判断も行います。
    # 本来はToolNodeへのルーティングなどもここで行うことになります。
    # messages に現在のステップの情報を追加してLLMに渡す
    step_execution_message = HumanMessage(content=f"現在の計画ステップは「{current_step_description}」です。このステップを実行してください。必要であればツールを使用してください。元のリクエストは「{state['input']}」でした。これまでの実行ステップ: {state['past_steps']}")
    
    # ヒント: llm_with_pne_tools を使って、現在のステップに関するメッセージでLLMを呼び出します。
    # state['messages'] に step_execution_message を追加したもので呼び出すのが良いでしょう。
    current_messages_for_step = state['messages'] + [step_execution_message]
    response = ____.invoke(current_messages_for_step)
    print(f"executor_node: LLM response for step: {response}")

    # AIMessageをmessagesに追加し、ToolCallがあればToolNodeに渡せるようにする
    # past_stepsにも今回の実行結果（の要約やステータス）を追加
    # ここでは簡単のため、LLMの応答をそのまま実行結果として記録
    updated_past_steps = past_steps + [(current_step_description, response.content if not response.tool_calls else "[Tool Call Triggered]")]
    
    return {
        "messages": [response], # LLMの応答（ToolCall含む可能性あり）をmessagesに追加
        "past_steps": updated_past_steps
    }

tool_node_pne = ToolNode(tools_pne)

# --- グラフ構築 ---
workflow_pne = StateGraph(PlanAndExecuteState)

workflow_pne.add_node("planner", planner_node)
workflow_pne.add_node("executor", executor_node)
workflow_pne.add_node("tools_pne", tool_node_pne)

workflow_pne.set_entry_point("planner")

workflow_pne.add_edge("planner", "executor")

# executor の後のルーティングロジック
def route_after_executor(state: PlanAndExecuteState):
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        print("route_after_executor: Routing to tools_pne")
        return "tools_pne"
    else:
        # ツールコールがない場合、計画の次のステップへ進むか、終了するか
        # ここでは、現在のステップが完了したとみなし、planから削除して再度executorへ
        if state["plan"]:
            print("route_after_executor: Step complete, proceeding to next step or finishing.")
            # 完了したステップをplanから削除
            # この更新はエグゼキュータ自身か、専用の更新ノードで行うべきだが、ここでは条件分岐内で行う
            remaining_plan = state["plan"][1:]
            # 状態を直接変更するのではなく、次のノードに渡す情報を返すのがLangGraphの基本
            # しかし、この条件分岐関数は次のノード名を返すだけなので、状態更新はノード側で行う必要がある。
            # Executorノードが次の呼び出しで新しいplanを見るように、planの更新はexecutorノードの責務とするか、
            # もしくは専用のplan更新ノードを挟む。
            # ここではexecutorがplanの先頭を見る前提なので、planが空になるまでループする。
            if not remaining_plan: # これ以上計画がなければ終了
                 print("route_after_executor: Plan finished, routing to END (via executor finalization).")
                 # executorが最終応答をresponseフィールドにセットする
                 return "executor" # executor に戻って最終処理 (response設定)
            else:
                 print("route_after_executor: More steps in plan, looping back to executor.")
                 # 実際にはここでplanを更新する操作が必要。executorがplanの先頭を見るため、
                 # executorの戻り値でplanを更新するか、専用ノードが必要。
                 # このサンプルではexecutorの冒頭でplanの最初の要素を処理し、
                 # route_after_executorでplanを更新してループする形を取るようにexecutorの戻り値を調整する。
                 # → executorの戻り値でplanを更新する形に修正するべき。
                 # この解答では、executorが自身の処理の最後にplanを更新すると仮定して進める。
                 # executorの最後に `return {"plan": state["plan"][1:], ...}` のような処理が必要になる。
                 # (この解答例ではexecutorの戻り値にplanの更新をまだ含めていないので注意)
                 # 簡単のため、executorは常にplanの0番目を見て、完了したらENDに行くようにする
                 # このため、現状はマルチステッププランには完全対応していない。
                 # Problem 005でこの部分を改善する。
                 # ここでは、ツールを使わない場合は「ステップ完了」とし、ENDに向かわせる。
                 # executorが最終応答をresponseフィールドにセットする。
                 return "executor" # executor に戻ってresponseを設定し、次の分岐でENDへ
        else:
            print("route_after_executor: No plan, routing to END (via executor finalization).")
            return "executor" # executor に戻って最終処理 (response設定)

workflow_pne.add_conditional_edges(
    "executor",
    route_after_executor,
    {
        "tools_pne": "tools_pne",
        "executor": "executor", # executor自身に戻る場合 (plan更新や最終応答生成のため)
        END: END # 本来はexecutorが最終的にENDに繋がるべきだが、route_after_executorで制御
    }
)

# ツール実行後、結果を持ってエグゼキューターに戻る
workflow_pne.add_edge("tools_pne", "executor")

app_pne = workflow_pne.compile()

# --- グラフの実行と結果表示 ---
print("\n--- Plan-and-Executeテスト: 日本の首都を調べて、その情報を元に何かをする (簡略版) ---")
# この入力では、プランナーは例えば「日本の首都を調べる」「その首都について何か言う」のような計画を立てることを期待
inputs_pne1 = {"input": "日本の首都はどこですか？そして、その都市はどんなところか簡単に教えてください。", "messages": []} 

for i, s in enumerate(app_pne.stream(inputs_pne1, {"recursion_limit": 10})):
    print(f"Stream output {i}: {s}")
    print("----")

final_state_pne1 = app_pne.invoke(inputs_pne1, {"recursion_limit": 10})
print(f"\nFinal Plan: {final_state_pne1.get('plan')}")
print(f"Past Steps: {final_state_pne1.get('past_steps')}")
print(f"Final Response: {final_state_pne1.get('response')}")
print(f"All Messages: {[m.pretty_repr() for m in final_state_pne1.get('messages', [])]}")

# 実行前にexecutorのplan更新ロジックの修正が必要なことに注意。
# このままではplanの最初のステップしか実行されないか、無限ループの可能性がある。
# 解答ではexecutorの最後にplanを更新する処理を追加する。


<details><summary>解答004</summary>

```python
# 解答004
from typing import TypedDict, Annotated, Sequence, List, Optional
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

# --- 状態定義 (PlanAndExecuteState) ---
class PlanAndExecuteState(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[tuple[str, str]], lambda x, y: x + y]
    messages: Annotated[Sequence[BaseMessage], add_messages]
    response: Optional[str]

# --- ツール定義 (問題003のget_capital_cityを再利用) ---
@tool
def get_capital_city(country_name: str) -> str:
    """Provides the capital city of a given country. 
    Only knows about Japan, France, and Canada."""
    print(f"get_capital_city tool called with: {country_name}")
    capitals = {
        "japan": "Tokyo",
        "france": "Paris",
        "canada": "Ottawa"
    }
    return capitals.get(country_name.lower(), f"Sorry, I don't know the capital of {country_name}.")

tools_pne = [get_capital_city]
llm_with_pne_tools = llm.bind_tools(tools_pne)

# --- プランナーノード ---
class Plan(BaseModel):
    steps: List[str] = Field(description="ユーザーのリクエストを達成するためのステップバイステップの計画")

def planner_node(state: PlanAndExecuteState):
    print(f"\nplanner_node: Generating plan for input: {state['input']}")
    structured_llm_planner = llm.with_structured_output(Plan)
    plan_prompt = (
        f"ユーザーリクエスト: {state['input']}\n"
        "上記のユーザーリクエストを達成するための計画をステップのリストとして生成してください。"
        "各ステップは具体的で実行可能なアクションであるべきです。"
        "例: 「日本の首都を調べる」、「フランスの首都について説明する」など。"
        "ツール「get_capital_city」が利用可能です。このツールは国名を引数に取り、首都名を返します。"
    )
    plan_result = structured_llm_planner.invoke(plan_prompt)
    print(f"planner_node: Generated plan: {plan_result.steps}")
    return {"plan": plan_result.steps, "past_steps": [], "messages": []} # messagesも初期化することが望ましい場合がある

# --- エグゼキューターノード ---
def executor_node(state: PlanAndExecuteState):
    plan = state["plan"]
    past_steps = state["past_steps"]

    if not plan: # 計画が空になったら終了処理
        print("executor_node: Plan is empty. Finalizing.")
        # 過去のステップを要約して最終応答とするか、最後のAIメッセージを使うなど
        summary_prompt = f"以下の計画と実行ステップに基づいて最終応答を生成してください:\nInput: {state['input']}\nExecuted Steps: {past_steps}"
        final_response_content = llm.invoke(summary_prompt).content
        return {"response": final_response_content, "messages": add_messages(state['messages'], [AIMessage(content=final_response_content)])}

    current_step_description = plan[0]
    remaining_plan = plan[1:]
    print(f"\nexecutor_node: Executing step: {current_step_description}")

    # 現在のメッセージ履歴に「これからこのステップを実行します」という情報を追加
    # これまでのメッセージ(`state['messages']`)と今回のステップ実行指示(`step_execution_message`)を結合
    step_execution_message = HumanMessage(
        content=f"Original request: '{state['input']}'.\nPrior plan steps results: {past_steps}.\nNow execute this step: '{current_step_description}'. Respond with the outcome of this step, or call a tool if needed."
    )
    # 既存のmessagesと新しいステップ指示を結合
    # add_messagesは状態更新時に使われるため、ここでは手動でリストを結合
    current_messages_for_llm = state['messages'] + [step_execution_message]

    llm_response_for_step = llm_with_pne_tools.invoke(current_messages_for_llm)
    print(f"executor_node: LLM response for step: {llm_response_for_step}")

    # 実行結果（LLMの直接応答 or ツール呼び出し指示）をmessagesに追加
    # past_stepsにも今回のステップと結果（LLM応答のcontent or ツール呼び出し）を記録
    step_outcome_summary = llm_response_for_step.content
    if llm_response_for_step.tool_calls:
        step_outcome_summary = f"[Tool Call: {llm_response_for_step.tool_calls[0]['name']}]"
        
    updated_past_steps = past_steps + [(current_step_description, step_outcome_summary)]

    return {
        "plan": remaining_plan, # 実行したステップをplanから削除
        "messages": add_messages(state['messages'], [step_execution_message, llm_response_for_step]), # 新しいメッセージを追加
        "past_steps": updated_past_steps
    }

tool_node_pne = ToolNode(tools_pne)

# --- グラフ構築 ---
workflow_pne = StateGraph(PlanAndExecuteState)

workflow_pne.add_node("planner", planner_node)
workflow_pne.add_node("executor", executor_node)
workflow_pne.add_node("tools_pne", tool_node_pne)

workflow_pne.set_entry_point("planner")
workflow_pne.add_edge("planner", "executor")

def route_after_executor(state: PlanAndExecuteState):
    last_message = state["messages"][-1] if state["messages"] else None
    if last_message and hasattr(last_message, "tool_calls") and last_message.tool_calls:
        print("route_after_executor: Routing to tools_pne")
        return "tools_pne"
    elif state["plan"]: # planが残っていれば再度executorへ
        print("route_after_executor: Plan has more steps, routing back to executor.")
        return "executor"
    else: # planが空なら終了（executorが最終応答をresponseにセット済みのはず）
        print("route_after_executor: Plan complete, routing to END.")
        return END

workflow_pne.add_conditional_edges(
    "executor",
    route_after_executor,
    {
        "tools_pne": "tools_pne",
        "executor": "executor",
        END: END
    }
)

workflow_pne.add_edge("tools_pne", "executor") # ツール実行後はエグゼキュータに戻ってplanを継続

app_pne = workflow_pne.compile()

# --- グラフの実行と結果表示 ---
print("\n--- Plan-and-Executeテスト: 日本の首都を調べて、その情報を元に何かをする ---")
inputs_pne1 = {"input": "日本の首都はどこですか？そして、その都市はどんなところか簡単に教えてください。", "messages": [], "plan": [], "past_steps": []}

for i, s in enumerate(app_pne.stream(inputs_pne1, {"recursion_limit": 10})):
    print(f"Stream output {i}: {s.get(list(s.keys())[0])}") # Print node output directly
    print("----")

final_state_pne1 = app_pne.invoke(inputs_pne1, {"recursion_limit": 10})
print(f"\nFinal Input: {final_state_pne1.get('input')}")
print(f"Final Plan: {final_state_pne1.get('plan')}")
print(f"Past Steps: {final_state_pne1.get('past_steps')}")
print(f"Final Response: {final_state_pne1.get('response')}")
# print(f"All Messages: {[m.pretty_repr() for m in final_state_pne1.get('messages', [])]}")

print("\n--- Plan-and-Executeテスト2: フランスの首都とその人口（ツール外情報）を調べる ---")
inputs_pne2 = {"input": "フランスの首都はどこですか？あと、その都市のおすすめ観光スポットを1つ教えて。", "messages": [], "plan": [], "past_steps": []}
for i, s in enumerate(app_pne.stream(inputs_pne2, {"recursion_limit": 10})):
    print(f"Stream output {i}: {s.get(list(s.keys())[0])}") # Print node output directly
    print("----")
final_state_pne2 = app_pne.invoke(inputs_pne2, {"recursion_limit": 10})
print(f"\nFinal Input: {final_state_pne2.get('input')}")
print(f"Final Plan: {final_state_pne2.get('plan')}")
print(f"Past Steps: {final_state_pne2.get('past_steps')}")
print(f"Final Response: {final_state_pne2.get('response')}")

```
</details>

<details><summary>解説004</summary>

#### この問題のポイント
*   **学習内容:**
    *   Plan-and-Execute（計画と実行）アーキテクチャの基本的な導入。
    *   状態(`PlanAndExecuteState`)に`input`、`plan`、`past_steps`、`response`といった計画実行のための新しいフィールドを追加する方法。
    *   LLMの構造化出力機能（`llm.with_structured_output(BaseModel)`）を利用して、テキストベースの計画を生成する「プランナー」ノードの作成。
    *   計画リストのステップを順番に処理する「エグゼキューター」ノードの概念。この問題では、エグゼキューターが計画の次のステップを取り出し、LLM（ツール利用可能）にそのステップの実行を指示し、計画を更新します。
    *   エグゼキューターからの分岐ロジック: ツール呼び出しが必要か、計画の次のステップに進むか、計画が完了して終了するかを判断する方法。
    *   複数ステップの計画を実行するためのループ構造の導入（エグゼキューターが自身に戻る）。
*   **コード解説:**
    *   **`PlanAndExecuteState`**: 
        *   `input`: ユーザーの最初の質問を保持します。
        *   `plan`: LLMによって生成された行動計画（文字列のリスト）を保持します。
        *   `past_steps`: 実行済みの計画ステップとその結果（文字列のタプルのリスト）を記録します。`Annotated[..., lambda x, y: x + y]`により、新しいステップ結果はリストに追加されます。
        *   `messages`: これまでの問題と同様に、LLMやツールとの対話履歴（特にツール呼び出し時）を保持します。
        *   `response`: エージェントによる最終的な応答文字列を保持します。
    *   **`Plan` (Pydantic Model)**: プランナーLLMが生成する計画の構造を定義します。`steps`という文字列リストを持つことを期待します。
    *   **`planner_node`**: 
        *   `llm.with_structured_output(Plan)` を使用して、LLMが`Plan`モデルに合致する形式で計画を出力するようにします。
        *   ユーザーの`input`に基づいて計画を生成し、`state.plan`を更新します。`past_steps`と`messages`を初期化します。
    *   **`executor_node`**: 
        *   `state.plan`が空の場合、計画完了とみなし、`past_steps`などに基づいて最終応答を生成し、`state.response`に設定します。
        *   `plan`にステップが残っている場合、最初のステップ (`current_step_description`) を取り出します。
        *   LLM (`llm_with_pne_tools`) に対し、現在のステップを実行するよう指示します。この際、元のリクエスト、過去のステップ結果、現在のステップ内容をコンテキストとして渡します。
        *   LLMの応答 (`llm_response_for_step`) を受け取り、これを`messages`に追加します。
        *   実行したステップとその結果の要約を`past_steps`に追加します。
        *   `plan`から実行済みステップを削除して更新します (`remaining_plan`)。
    *   **`tool_node_pne`**: 問題003までと同様の`ToolNode`ですが、Plan-and-Execute用のツールセットを使用します。
    *   **グラフのルーティング (`route_after_executor`)**:
        *   エグゼキューターの実行後、最新のメッセージにツールコールがあれば `tools_pne` へルーティングします。
        *   ツールコールがなく、`plan` にまだステップが残っていれば、再度 `executor` に戻り、次のステップを実行します。
        *   ツールコールがなく、`plan` も空であれば（全ステップ完了）、`END` にルーティングします。
    *   **`app_pne.stream` / `app_pne.invoke`**: `recursion_limit`を適切に設定して、複数ステップの計画実行に対応します。

この問題では、エージェントが単に反応するだけでなく、複数ステップの計画を立ててそれを逐次実行していく、より高度な動作の基礎を構築します。プランナーとエグゼキューターの分離は、複雑な問題解決能力向上のための重要なパターンです。次の問題では、このPlan-and-Executeの仕組みをさらに洗練させていきます。
---
</details>

In [ ]:
# 今後の実装に向けた空のコードセル